在TinyStories上训练以字节为单位的BPE分词器。

##### 预分词的并行化
预分词步骤将成为一个重要的性能瓶颈，故可以使用built-in library `multiprocessing`并行化来加速。

在并行化预分词的实现中，将语料分块时要注意块边界出现在special token的开始处。提供的预分词示例代码可以用于找到块的边界，找到后即可用于并行时的任务分配。这种分块策略永远是有用的，因为我们永远不会希望跨文档的合并操作。本作业中无需担心语料中没有`<|endoftext|>`导致块过大。

##### 去掉special token
在用`re.finditer`正则预分词之前，去掉special token（无论你处理整个语料还是某个块）。

使用`re.split`和` "|" ⌋.join(special_tokens)`。(with careful use of re.escape since | may occur in the special tokens)

这部分对应`test_train_bpe_special_tokens`。

##### 优化合并步骤
最朴素的合并算法太慢了，因为每次合并都要去看所有当前的字节对（或token对）。However, the only pair counts that change after each merge are those that overlap with the merged pair. Thus, BPE training speed can be improved by indexing the counts of all pairs and incrementally updating these counts, 而非一直遍历并计数所有对。这样能快很多，尽管这里不能并行。

##### 对于低算力：Profiling
用`cProfile`或`scalene`等工具分析性能瓶颈并优化它们。

##### 对于低算力：Downscaling/降尺度
先在数据集的一小部分上实验，例如在验证集上训练，大小约百分之一。选取小的子集时也要注意不能太小。

### Problem (train_bpe): BPE Tokenizer Training (15 points)
交付内容：写一个函数，输入文本文件路径，训练字节为单位的bpe分词器。

输入参数：
- `input_path`：`str`字符串，数据文本文件路径
- `vocab_size`：`int`正整数，最终词表大小，包含最初的各种字节、合并出的内容、special tokens
- `special_tokens`：`list[str]`字符串列表，不影响bpe训练

返回参数：
- `vocab`：`dict[int,bytes]`字节串的词典映射，每个字节串（即最终得到的token）编一个整数序号（即token ID）。
- `merges`：`list[tuple[bytes, bytes]]`字节串二元组的列表，训练过程中合并token的记录，按合并顺序从前往后列出。

最终目标：将写好的内容填入`adapters.py`文件中`run_train_bpe`函数处，运行`uv run pytest tests/test_train_bpe.py`进行测试。

附加可选目标：把关键部分用cpp（用cppyy）或rust（用PyO3）来写。如果你要这么做，请注意区分哪些操作需要复制内存，哪些是直接从 Python 内存中读取。另外，请务必留下构建说明，或者确保项目仅使用 `pyproject.toml` 文件就能完成构建。

另外注意给定的GPT-2的正则模板未必所有引擎中都支持，即使支持可能也很慢。已经验证`Oniguruma`库的速度相当快，并且支持负向先行断言（negative lookahead），但Python的`regex`也并不逊色。

In [1]:
from pprint import pprint
from cs336_basics.pretokenization_example import *
import regex as re
from IPython.display import clear_output, display

In [2]:
def path_to_chunks_bytes(p:str, n_parallel: int) -> list[bytes] :
    res = []
    with open(p,"rb") as f:
        num_processes = n_parallel
        boundaries = find_chunk_boundaries(f, num_processes, b"<|endoftext|>")
        for start, end in zip(boundaries[:-1], boundaries[1:]):
            f.seek(start)
            chunk = f.read(end - start) #.decode("utf-8", errors="ignore")
            res.append(chunk)
    return res

testchunks = path_to_chunks_bytes(p="../data/TinyStoriesV2-GPT4-valid.txt",n_parallel=4)
for i, chunk in enumerate(testchunks):
    print(f"第{i}段的前100个字节：")
    print(chunk[:100])


第0段的前100个字节：
b'u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little '
第1段的前100个字节：
b'<|endoftext|>\nOnce upon a time, in a green forest, there lived a big bear and a little bunny. They w'
第2段的前100个字节：
b'<|endoftext|>\nOnce upon a time, there was a little boy named Tim. Tim loved to print pictures of ani'
第3段的前100个字节：
b'<|endoftext|>\nOnce upon a time, there was a big purple cat named Tom. Tom lived in a little house wi'


In [3]:
# 对于每段文本，去掉特殊token并切分的过程
def pre_tokenization_for_chunk(text_chunk_bytes:bytes, special_tokens: list[str]) -> list[str]:
    text_chunk = text_chunk_bytes.decode("utf-8")
    escaped_special_tokens = [re.escape(t) for t in special_tokens]
    escaped_special_tokens_in_one_str = "|".join(escaped_special_tokens)
    # print(escaped_special_tokens_in_one_str)
    splited_text = re.split(escaped_special_tokens_in_one_str,text_chunk)
    pre_tokenization = []
    PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
    for doc in splited_text:
        it = re.finditer(PAT,doc)
        for match in it:
            pre_tokenization.append(match.group())
    return pre_tokenization

testpretok = []
for i, chunk in enumerate(testchunks) : 
    print(f"------第{i}段测试文本信息：------")
    print("前100字符：",chunk[:100])
    res = pre_tokenization_for_chunk(chunk,["<|endoftext|>"])
    print("预分词结果前10个：")
    print(res[:10])
    testpretok.append(res)

# print("总体预分词结果：",testpretok[:100])

------第0段测试文本信息：------
前100字符： b'u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little '
预分词结果前10个：
['u', ' don', "'t", ' have', ' to', ' be', ' scared', ' of', ' the', ' loud']
------第1段测试文本信息：------
前100字符： b'<|endoftext|>\nOnce upon a time, in a green forest, there lived a big bear and a little bunny. They w'
预分词结果前10个：
['\n', 'Once', ' upon', ' a', ' time', ',', ' in', ' a', ' green', ' forest']
------第2段测试文本信息：------
前100字符： b'<|endoftext|>\nOnce upon a time, there was a little boy named Tim. Tim loved to print pictures of ani'
预分词结果前10个：
['\n', 'Once', ' upon', ' a', ' time', ',', ' there', ' was', ' a', ' little']
------第3段测试文本信息：------
前100字符： b'<|endoftext|>\nOnce upon a time, there was a big purple cat named Tom. Tom lived in a little house wi'
预分词结果前10个：
['\n', 'Once', ' upon', ' a', ' time', ',', ' there', ' was', ' a', ' big']


In [4]:
def get_all_pretoken_bytes_and_build_count_dict(pre_tokens:list[list[str]]) -> dict[bytes,int]:
    res = {}
    for trunks in pre_tokens:
        for tok in trunks:
            btok = tok.encode("utf-8")
            res[btok] = res.get(btok,0) + 1
    return res

pre_tokens_bytes_dict = get_all_pretoken_bytes_and_build_count_dict(testpretok)
print("整合所有预分词结果并转为bytes，前10个：")
pprint(list(pre_tokens_bytes_dict.keys())[:10])

整合所有预分词结果并转为bytes，前10个：
[b'u',
 b' don',
 b"'t",
 b' have',
 b' to',
 b' be',
 b' scared',
 b' of',
 b' the',
 b' loud']


In [5]:
pre_token_ints_dict = {tuple(pre_token):cnt for pre_token , cnt in pre_tokens_bytes_dict.items()}
print("测试文本转为整数组，前十个pretoken")
pprint(list(pre_token_ints_dict.keys())[:10])

测试文本转为整数组，前十个pretoken
[(117,),
 (32, 100, 111, 110),
 (39, 116),
 (32, 104, 97, 118, 101),
 (32, 116, 111),
 (32, 98, 101),
 (32, 115, 99, 97, 114, 101, 100),
 (32, 111, 102),
 (32, 116, 104, 101),
 (32, 108, 111, 117, 100)]


In [6]:
type IntsCount = dict[tuple[int,...],int]
type IntPairInInts = dict[tuple[int,int],set[tuple[int,...]]]

def build_inverted_index(pre_token_ints_dict:IntsCount) -> IntPairInInts:
    inverted_index = {}
    for key in pre_token_ints_dict.keys():
        pairs = zip(key[:-1],key[1:])
        for pair in pairs:
            inverted_index.setdefault(pair,set()).add(key)
    return inverted_index

testinvindex = build_inverted_index(pre_token_ints_dict)
pprint(list(testinvindex.keys())[:10])

[(32, 100),
 (100, 111),
 (111, 110),
 (39, 116),
 (32, 104),
 (104, 97),
 (97, 118),
 (118, 101),
 (32, 116),
 (116, 111)]


In [7]:
class BpeManager:
    data: IntsCount
    inv_index: IntPairInInts
    vocab_dict: dict[int,bytes]
    merge_list:list[tuple[bytes,bytes]]

    def __init__(self, pre_token_ints_dict:IntsCount, special_tokens:list[str]) -> None:
        self.data = pre_token_ints_dict
        self.inv_index = build_inverted_index(pre_token_ints_dict=pre_token_ints_dict)
        self.vocab_dict = {n: bytes([n]) for n in range(256)}
        for st in special_tokens:
            self.vocab_dict[len(self.vocab_dict)] = st.encode("utf-8")
        self.merge_list = []
    
    # 单个单词中数出现次数
    def get_int_pair_occur_in_ints(self,intpair:tuple[int,int], ints: tuple[int,...]) -> int:
        count = 0
        for i in range(len(ints) - 1):
            if ints[i] == intpair[0] and ints[i+1] == intpair[1] :
                count += 1
        return count

    # 多个单词中数出现次数
    def get_int_pair_occur_times(self,intpair:tuple[int,int]) -> int:
        sum = 0
        for pretokints in self.inv_index[intpair]:
            sum += self.get_int_pair_occur_in_ints(intpair,pretokints) * self.data[pretokints]
        return sum

    def get_max_token_pair_int(self) -> tuple[int,int]:
        maxpair = max(
            self.inv_index, 
            key = lambda k : (                          # 这里k代指self.inv_index的key中的每个元素，即一个整数对
                self.get_int_pair_occur_times(k),       # 先按其出现次数排
                self.vocab_dict[k[0]],                  # 再看第一位的bytes
                self.vocab_dict[k[1]]                   # 最后看第二位的bytes
            )
        )
        # print("---寻找出现最多的token对---\n",
        #     f"出现最多的token对是{maxpair}，即{self.vocab_dict[maxpair[0]]}与{self.vocab_dict[maxpair[1]]}\n",
        #     f"出现了{self.get_int_pair_occur_times(maxpair)}次\n",
        #     f"出现在这些pretoken中（前10个）：{list(self.inv_index[maxpair])[:10]}\n",
        #     "---寻找结束---\n")
        return maxpair

    def build_new_token(self,new_pair_id:tuple[int,int]) -> tuple[int,bytes,int,bytes,int,bytes]:
        # 在vocab_dict中加入新token nt = lt + rt
        # 在merge中加入新merge (lt,rt)
        li = new_pair_id[0]
        ri = new_pair_id[1]
        lt = self.vocab_dict[li]
        rt = self.vocab_dict[ri]
        ni = len(self.vocab_dict)
        nt = lt + rt
        self.vocab_dict[ni] = nt
        self.merge_list.append((lt,rt))
        return (li,lt,ri,rt,ni,nt)

    # 一次merge的全过程
    def merge(self, new_pair_id:tuple[int,int]) -> None:
        (li,lt,ri,rt,ni,nt) = self.build_new_token(new_pair_id=new_pair_id)
        # print("---开始合并---\n",
        #     "本次合并情况：\n",
        #     f"{li}--{lt}与{ri}--{rt}合并得到{nt}，编号为{ni}")
        
        # 从inv_index中找到所有包含 (li,ri) 的pretoken单词
        # assert (li,ri) in self.inv_index , "反向索引中未找到所合并的对"
        # print("他们出现在这些单词中（前10个）：",f"{list(self.inv_index[(li,ri)])[:10]}")

        # 合并过程
        # 两件事：维护反向索引，更新data
        # 注意：原始的data中的每种pretoken是必定互不相同的，并且永远不可能变得相同
        # 实际上，同一序列是有可能拆分为两种不同的token的，例如cat = c + at = ca + t，但是在bpe中永远不需要考虑这种事情

        # 取出(li,ri)对应的所有pretoken(...,xi,li,ri,yi,...)
        #     对于每个pretoken(...xi,li,ri,yi,...)：
        #         取出所有它包含的pair(zi,wi)（包括(li,ri)本身）
        #         对于每个pair(zi,wi)：
        #             找到它里面的那个pretoken(...,xi,li,ri,yi,...)
        #             （即使这个pair对应的其他单词里也有(li,ri)，也没事，因为总会遍历到的）
        #             删掉它！
        #         新建一个pretoken(...,xi,ni,yi,...)，加入语料库，次数与原本相同
        #         取出所有它包含的pair(zi,wi)
        #         对于每个pair(zi,wi)：
        #             加上(...,xi,ni,yi,...)
        #         从语料库中删除这个pretoken(...xi,li,ri,yi,...)
        #     删掉这个(li,ri)（删前确保里面所有的单词都已经被删掉了）

        while self.inv_index[(li,ri)] :
            nowpretok = self.inv_index[(li,ri)].pop()
            n = self.data[nowpretok]
            del self.data[nowpretok]
            for pair in zip(nowpretok[:-1],nowpretok[1:]):
                self.inv_index[pair].discard(nowpretok)
            newpretok = self.get_new_pretok_from_oldpretok_and_pair(nowpretok,(li,ri),ni=ni)
            self.data[newpretok] = n
            for pair in zip(newpretok[:-1],newpretok[1:]):
                self.inv_index.setdefault(pair,set()).add(newpretok)

        # 清理inv_index中空的pair
        for pair in list(self.inv_index.keys()):
            if self.inv_index[pair] == set():
                del self.inv_index[pair]

        # print("---合并结束---\n")

    def get_new_pretok_from_oldpretok_and_pair(self,oldpretok:tuple[int,...],pair:tuple[int,int],ni:int) -> tuple[int,...]:
        # 注意处理pair出现多次的情况！
        newpretok = ()
        i = 0
        while i < len(oldpretok):
            if i < len(oldpretok) - 1 and (oldpretok[i],oldpretok[i+1]) == pair:
                # 找到一个pair
                newpretok += (ni,)
                i += 2
            else:
                newpretok += (oldpretok[i],)
                i += 1
        return newpretok

    def quick_look(self):
        pass
        # print("------状态速览------")
        # print("当前的data：")
        # print(self.data)
        # print([bytes(pt) for pt in self.data])
        # print([[self.vocab_dict[i] for i in j] for j in self.data])
        # print("当前的token对位置索引inv_index：")
        # print(self.inv_index)
        # print([[(self.vocab_dict[k[0]],self.vocab_dict[k[1]]),v] for k,v in self.inv_index.items()])
        clear_output(wait=True)
        display(f"当前词表大小：{len(self.vocab_dict)}，最新token：{list(self.vocab_dict.items())[-1:]}")
        # print("当前的合并列表：")
        # print(self.merge_list)
        # print("------状态速览------")
        # print()


In [8]:

def main_bpe(
    pre_token_ints_dict: IntsCount,
    vocab_size: int,
    special_tokens: list[str]
    )->tuple[
        dict[int,bytes],                # vocab
        list[tuple[bytes,bytes]]        # merge
    ]:

    manager = BpeManager(pre_token_ints_dict=pre_token_ints_dict, special_tokens=special_tokens)

    while len(manager.vocab_dict) < vocab_size:
        
        manager.quick_look()
        if manager.inv_index == {}:
            print("不再有任何相邻token对，合并过程提前结束")
            break
        maxpair = manager.get_max_token_pair_int()
        # print("获得的最大token对：",
        #       manager.vocab_dict[maxpair[0]],
        #       manager.vocab_dict[maxpair[1]]
        #       )
        manager.merge(maxpair)

    return (manager.vocab_dict, manager.merge_list)


(vocab,merge) = main_bpe(pre_token_ints_dict=pre_token_ints_dict, vocab_size=10000, special_tokens=["<|endoftext|>"])

"当前词表大小：9999，最新token：[(9998, b' raincoat')]"

In [9]:
print(list(vocab.items())[-10:])
print(merge[-10:])

[(9990, b' sly'), (9991, b' sinking'), (9992, b' roles'), (9993, b' roamed'), (9994, b' ribbit'), (9995, b' respecting'), (9996, b' releasing'), (9997, b' refusing'), (9998, b' raincoat'), (9999, b' racers')]
[(b' s', b'ly'), (b' s', b'inking'), (b' ro', b'les'), (b' ro', b'amed'), (b' ribb', b'it'), (b' res', b'pecting'), (b' rele', b'asing'), (b' ref', b'using'), (b' rain', b'coat'), (b' rac', b'ers')]


In [10]:
def my_bpe(input_path:str, vocab_size:int, special_tokens:list[str]):
    # 从路径读取文件，转为文件指针。注意读出来的是bytes类型
    chunks_bytes = path_to_chunks_bytes(p=input_path, n_parallel=4)
    # 对每段文本进行预分词
    pre_tokens = []
    for chunk in chunks_bytes:
        pretok = pre_tokenization_for_chunk(chunk, special_tokens)
        pre_tokens.append(pretok)
    # 整合所有预分词结果并转为bytes类型的token，建立词频字典
    pre_tokens_bytes_dict = get_all_pretoken_bytes_and_build_count_dict(pre_tokens)
    # 转为整数组表示的token，建立词频字典
    pre_token_ints_dict = {tuple(pre_token):cnt for pre_token , cnt in pre_tokens_bytes_dict.items()}
    # 运行BPE算法
    (vocab,merge) = main_bpe(pre_token_ints_dict=pre_token_ints_dict, vocab_size=vocab_size, special_tokens=special_tokens)

    return (vocab,merge)
    

    

(vocab,merge) = my_bpe("../data/TinyStoriesV2-GPT4-valid.txt", 10000, ["<|endoftext|>"])
print("最终结果展示：")
print("词表大小：",len(vocab))
print("词表最后10个：",list(vocab.items())[-10:])
print("合并列表最后10个：",merge[-10:])

"当前词表大小：9999，最新token：[(9998, b' raincoat')]"

最终结果展示：
词表大小： 10000
词表最后10个： [(9990, b' sly'), (9991, b' sinking'), (9992, b' roles'), (9993, b' roamed'), (9994, b' ribbit'), (9995, b' respecting'), (9996, b' releasing'), (9997, b' refusing'), (9998, b' raincoat'), (9999, b' racers')]
合并列表最后10个： [(b' s', b'ly'), (b' s', b'inking'), (b' ro', b'les'), (b' ro', b'amed'), (b' ribb', b'it'), (b' res', b'pecting'), (b' rele', b'asing'), (b' ref', b'using'), (b' rain', b'coat'), (b' rac', b'ers')]


课程提供的pretokenization用法
```python
with open(..., "rb") as f:
    num_processes = 4
    boundaries = find_chunk_boundaries(f, num_processes, b"<|endoftext|>")

    # The following is a serial implementation, but you can parallelize this
    # by sending each start/end pair to a set of processes.
    for start, end in zip(boundaries[:-1], boundaries[1:]):
        f.seek(start)
        chunk = f.read(end - start).decode("utf-8", errors="ignore")
        # Run pre-tokenization on your chunk and store the counts for each pre-token
```

### Problem (train_bpe_tinystories): BPE Training on TinyStories (2 points)
**(a) 在 TinyStories 数据集上训练字节级 BPE 分词器。词表大小为10000。TinyStories的special token是<|endoftext|>。将训练生成的词表和合并序列化（serialize，即存成一个json之类的文件）到本地。训练过程花费了多少小时，占用了多少内存？词表中最长的词元（token）是什么？这个结果是否合理？**

资源限制：不使用GPU情况下不超过30分钟，不超过30GB内存。

提示：注意`<|endoftext|>`分割了各个文档。在进行bpe合并前要先处理它们。知道以上事实并使用并行预分词可以将时间压缩到两分钟内。

交付内容：一到两句话。

**(b)分析代码性能，哪一步耗时最多？** 交付内容：一到两句话。

下面再在 `OpenWebText` 上训练试试。

### Problem (train_bpe_expts_owt): BPE Training on OpenWebText (2 points)
**(a) 同上题，数据集改为OpenWebText，词表大小改为32000。**

资源限制：不使用GPU情况下不超过12小时，不超过100GB内存。

交付内容：一到两句话。

**(b) 比较两个数据集上训练的分词器的区别。** 交付内容：一到两句话。